In [3]:
!pip install lazypredict

/usr/bin/zsh: /home/roberto/.conda/envs/tensorflow-env/lib/libncursesw.so.6: no version information available (required by /usr/bin/zsh)


In [6]:
import lazypredict
import pandas as pd
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

In [8]:
data = pd.read_csv("datasets/edited/fasttext_embedded_davidson.csv")
del data["Unnamed: 0"]
data.head()

,0,1,2,3,4,5,6,7,8,9,...,93,94,95,96,97,98,99,positive,neutral,negative
0,0.049839,-0.012389,-0.023938,0.020247,0.039021,0.000894,-0.035659,0.014219,-0.007426,-0.001875,...,0.037399,-0.046555,0.007986,0.050923,0.043329,0.003237,0.004343,0.0679,0.4113,0.5208
1,-0.036568,0.003346,0.033397,0.013426,0.030098,0.017432,-0.029294,-0.018229,0.038597,0.026475,...,-0.016421,0.034347,-0.006825,0.049985,0.011514,0.028711,0.029309,0.0326,0.4309,0.5365
2,-0.033726,-0.026513,0.051228,0.023314,-0.008450,-0.007304,-0.002611,-0.044960,-0.021767,0.029252,...,-0.018170,0.007356,0.025731,-0.022734,-0.031255,0.010993,0.006138,0.0031,0.0268,0.9701
3,0.016069,-0.023210,0.017003,0.020582,0.020618,-0.082636,0.031038,-0.041252,-0.050100,-0.016178,...,0.071336,-0.090715,0.037946,-0.038263,0.062424,0.014616,-0.024076,0.0609,0.7310,0.2081
4,-0.039872,0.003142,-0.052659,-0.010374,0.019240,-0.054540,0.042485,0.009805,-0.032044,0.017162,...,0.019592,0.058020,0.027119,-0.024716,-0.033323,-0.031128,0.028149,0.0119,0.1853,0.8027
